In [1]:
import re
import os
import pandas as pd

# Diccionario de monedas
monedas_dict = {
    # Euro
    "euro": "EUR", "euros": "EUR", "eur": "EUR", "€": "EUR",
    
    # Yen japonés
    "yen": "JPY", "yenes": "JPY", "yenes japoneses": "JPY", "jpy": "JPY", "jp": "JPY", "¥": "JPY",
    
    # Dólar estadounidense
    "dólar": "USD", "dolar": "USD", "dólares": "USD", "dolares": "USD",
    "usd": "USD", "us$": "USD", "u$s": "USD", "$": "USD",
    "dólares estadounidenses": "USD", "dolares estadounidenses": "USD",
    "dólares americanos": "USD", "dolares americanos": "USD",
    "dólar americano": "USD", "dolar americano": "USD",
    
    # Dólar australiano
    "dólar australiano": "AUD", "dolar australiano": "AUD",
    "dólares australianos": "AUD", "dolares australianos": "AUD",
    "australiano": "AUD", "australianos": "AUD", "aud": "AUD",
    
    # Libra esterlina
    "libra": "GBP", "libras": "GBP", "libras esterlinas": "GBP",
    "gbp": "GBP", "£": "GBP",
    
    # Franco suizo
    "franco": "CHF", "francos": "CHF", "francos suizos": "CHF", "franco suizo": "CHF", "chf": "CHF",
    
    # Dólar canadiense
    "dólar canadiense": "CAD", "dolar canadiense": "CAD",
    "dólares canadienses": "CAD", "dolares canadienses": "CAD",
    "cad": "CAD", "canadiense": "CAD", "canadienses": "CAD",
    
    # Peso mexicano
    "peso mexicano": "MXN", "pesos mexicanos": "MXN",
    "mxn": "MXN", "peso": "MXN", "pesos": "MXN",
    
    # Real brasileño
    "real brasileño": "BRL", "reales brasileños": "BRL", "brl": "BRL",
    
    # Yuan chino
    "yuan": "CNY", "yuanes": "CNY", "cny": "CNY", "renminbi": "CNY", "rmb": "CNY", "元": "CNY",
    
    # Peso chileno
    "peso chileno": "CLP", "pesos chilenos": "CLP", "clp": "CLP",
    
    # Peso colombiano
    "peso colombiano": "COP", "pesos colombianos": "COP", "cop": "COP",
    
    # Sol peruano
    "sol": "PEN", "soles": "PEN", "pen": "PEN", "sol peruano": "PEN", "soles peruanos": "PEN",
    
    # Peso argentino
    "peso argentino": "ARS", "pesos argentinos": "ARS", "ars": "ARS",
    
    # Rublo ruso
    "rublo": "RUB", "rublos": "RUB", "rub": "RUB", "₽": "RUB",
    
    # Rupia india
    "rupia": "INR", "rupias": "INR", "inr": "INR", "₹": "INR",
    
    # Rand sudafricano
    "rand": "ZAR", "rands": "ZAR", "zar": "ZAR",
    
    # Won surcoreano
    "won": "KRW", "wones": "KRW", "krw": "KRW", "₩": "KRW",
}


def normalizar_numero(num_str: str) -> float:
    """Convierte formatos como 3.980.64 o 16,073 en float correcto"""
    s = num_str.strip().replace(" ", "")
    s = s.replace(",", ".")
    if s.count(".") > 1:
        partes = s.split(".")
        s = "".join(partes[:-1]) + "." + partes[-1]
    s = re.sub(r"[^\d.]", "", s)
    return float(s)


def extraer_nombre(texto: str) -> str | None:
    """Extrae únicamente el primer nombre de la persona al inicio de la llamada"""
    patron = re.search(
        r"(?:soy|le habla|habla|al habla|le saluda|les saluda)\s+([A-ZÁÉÍÓÚÑ][a-záéíóúñ]+)",
        texto,
        re.IGNORECASE
    )
    if patron:
        nombre = patron.group(1).strip().capitalize()
        # Eliminar coletillas como "de Forex Capital..."
        nombre = re.sub(r"\s+de\s+.*", "", nombre, flags=re.IGNORECASE)
        return nombre
    return None


# Carpeta con transcripciones
carpeta = r"C:\Users\onnyx\Desktop\Audios\Texto\TranscripcionesH"

# Lista para acumular resultados
registros = []

# Recorrer todos los archivos .txt
for archivo in os.listdir(carpeta):
    if archivo.lower().endswith(".txt"):
        ruta = os.path.join(carpeta, archivo)

        # Extraer id del nombre del archivo
        try:
            id_llamada = int(os.path.splitext(os.path.basename(ruta))[0].split("_")[1])
        except Exception:
            id_llamada = os.path.splitext(os.path.basename(ruta))[0]

        # Leer texto
        with open(ruta, "r", encoding="utf-8") as f:
            texto = f.read()

        # Buscar nombre (mejorado, solo primer nombre)
        nombre = extraer_nombre(texto)

        # Tipo de operación
        if re.search(r"\bcomprar\b", texto, re.IGNORECASE):
            tipo_operacion = "comprar"
        elif re.search(r"\bvender\b", texto, re.IGNORECASE):
            tipo_operacion = "vender"
        else:
            tipo_operacion = None

        # Extraer todos los números con decimales
        nums_raw = re.findall(r"\d{1,3}(?:[.,]\d{3})*[.,]\d+", texto)
        nums = [normalizar_numero(n) for n in nums_raw]

        # Asignar valores según la posición
        monto_numerico = nums[0] if len(nums) > 0 else None
        tasa_inicial = nums[1] if len(nums) > 1 else None
        tasa_final = nums[-1] if len(nums) > 2 else None
        spread = round(abs(tasa_inicial - tasa_final), 2) if (tasa_inicial and tasa_final) else None

        # Detectar moneda
        moneda = None
        for desc, code in monedas_dict.items():
            if re.search(desc, texto, re.IGNORECASE):
                moneda = code
                break

        # Guardar registro
        registros.append({
            "id": id_llamada,
            "nombre": nombre,
            "tipo_operacion": tipo_operacion,
            "monto_numerico": monto_numerico,
            "tasa_inicial": tasa_inicial,
            "tasa_final": tasa_final,
            "moneda": moneda,
            "spread": spread,
            "texto_llamada": texto.strip()
        })

# Convertir a DataFrame con todos los archivos
df = pd.DataFrame(registros)
df

,id,nombre,tipo_operacion,monto_numerico,tasa_inicial,tasa_final,moneda,spread,texto_llamada
0,1,Sofía,comprar,16.073,3980.64,3956.12,USD,24.52,"Buenos días, soy Sofía de Forex Capital, podem..."
1,10,Laura,vender,52.898,4431.31,4414.91,JPY,16.40,"Muy buenos días, le habla Laura de Forex Capit..."
2,100,Sara,comprar,80.951,4261.25,4239.38,USD,21.87,"Hola, soy Sara Gómez, representante de Forex C..."
3,101,Sofía,comprar,89.228,4091.37,4064.83,USD,26.54,"Hola, habla Sofía de Forex Capital. Le interes..."
4,102,Laura,vender,71.042,3914.56,3879.36,USD,35.20,"Hola, soy Laura de Forex Capital. ¿Está dispon..."
...,...,...,...,...,...,...,...,...,...
495,95,None,vender,63.264,4297.95,253.03,USD,4044.92,"Buen día, aquí Sara Márquez de Forex Capital, ..."
496,96,Sara,comprar,82.704,4374.15,356.44,USD,4017.71,"Hola, le saluda Sara Márquez del Departamento ..."
497,97,Sara,comprar,77.603,3996.87,3971.85,JPY,25.02,"Hola, habla Sara Gómez de Forex Capital. Le in..."
498,98,Juliana,comprar,35.312,4122.79,88.46,USD,4034.33,Les saluda Juliana de Forex Capital. Estamos o...


In [2]:
# Exportar a Excel
salida_excel = os.path.join(carpeta, "resultado_transcripciones.xlsx")
df.to_excel(salida_excel, index=False, engine="openpyxl")
print(f"Archivo exportado en: {salida_excel}")

Archivo exportado en: C:\Users\onnyx\Desktop\Audios\Texto\TranscripcionesH\resultado_transcripciones.xlsx
